In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from pandas import read_csv, DataFrame

## Config Vars

In [36]:
conf_test_dataset_path = 'data/test_dataset.csv'
shift_predictions_path = 'data/shift_predictions'
model_type = 'gru'
conf_dataset_columns_to_drop = ["@timestamp"]
shifts = 3

## Load Dataset

In [37]:
train_dataset = read_csv(conf_test_dataset_path , header=0, index_col=0)
train_dataset_bck = read_csv(conf_test_dataset_path , header=0, index_col=0)
train_dataset.drop(conf_dataset_columns_to_drop, axis=1, inplace=True)
train_dataset[train_dataset.columns] = StandardScaler().fit_transform(train_dataset)
train_dataset_no_anotation = train_dataset.drop("label", axis=1)

In [38]:
# train_dataset = read_csv(conf_test_dataset_path , header=0, index_col=0)
train_dataset_bck.head()

,@timestamp,system.cpu.user.pct,system.cpu.system.pct,system.cpu.idle.pct,system.cpu.iowait.pct,system.cpu.softirq.pct,system.cpu.total.pct,system.memory.used.pct,system.network.in.bytes,system.network.in.packets,...,system.diskio.iostat.write.per_sec.bytes,jvm.metrics.memory.heap_memory_usage.committed,jvm.metrics.memory.heap_memory_usage.max,jvm.metrics.memory.heap_memory_usage.used,jvm.metrics.threading.thread_count,jvm.metrics.gc.psms.collection_count,jvm.metrics.gc.psms.collection_time,jvm.metrics.gc.pss.collection_count,jvm.metrics.gc.pss.collection_time,label
57572,2018-01-07 15:55:20,0.1933,0.0627,3.7207,0.0111,0.0121,0.2793,0.9717,4000000.0,7859.0,...,57782.0,2.053112e+09,2.053112e+09,865614728.0,1001.0,0,0,2,85,0
57573,2018-01-07 15:55:30,0.2531,0.0623,3.6334,0.0372,0.0141,0.3666,0.9721,4000000.0,7537.0,...,57782.0,2.053112e+09,2.053112e+09,865614728.0,1001.0,0,0,2,85,0
57574,2018-01-07 15:55:40,0.2104,0.0455,3.7269,0.0030,0.0142,0.2731,0.9721,4000000.0,7730.0,...,57782.0,2.053112e+09,2.053112e+09,865614728.0,1001.0,0,0,2,85,0
57575,2018-01-07 15:55:50,0.2084,0.0516,3.6964,0.0324,0.0111,0.3036,0.9721,4000000.0,7298.0,...,57782.0,2.053112e+09,2.053112e+09,865614728.0,1001.0,0,0,2,85,0
57576,2018-01-07 15:56:00,0.2854,0.0724,3.6211,0.0040,0.0171,0.3789,0.9723,4000000.0,8029.0,...,60525.0,2.050490e+09,2.050490e+09,972126736.0,1082.0,0,0,2,84,0


## Load Predctions

In [39]:
shift_predictions = []
for shift in range(1, shifts+1):
    dset = read_csv(shift_predictions_path + '/predicted_data_' + model_type + '_model_' + str(shift) + '.csv' , header=0, index_col=0)
    shift_predictions.append(dset)

## Calculating error vector (Using MSE)
e (t) = [e<sub>11</sub><sup>(t)</sup>, ...,e<sub>1l</sub><sup>(t)</sup>, ...,e<sub>d1</sub><sup>(t)</sup>, ...,e<sub>dl</sub><sup>(t)</sup>]

In [40]:
error_vects = DataFrame(columns = [1,2,3])
for row in range(5, train_dataset.shape[0]):
    vect = []
    for shift in range(shifts):
        error = mean_squared_error(shift_predictions[shift].iloc[[row - (shift + 1)]],train_dataset_no_anotation.iloc[[row]])
        vect.append(error)
    error_vects.loc[row] = vect 
    

In [41]:
error_vects.index = train_dataset.index[5:]
error_vects[["label"]] = train_dataset_bck[["label"]][5:]

In [42]:
error_vects.to_csv("data/error_vects.csv")